In [ ]:
# Установим библиотеку для работы с датасетами на kaggle
!pip install opendatasets --quiet

In [ ]:
import opendatasets as od

# Загрузим датасет на прямую с kaggle
dataset_url = 'https://www.kaggle.com/competitions/star-type-classification'

# Upload Kaggle.json file to Colab… (Caution: Do not Share kaggle.json as contains API key of your Kaggle Account)

od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ignatikz
Your Kaggle Key: ··········


100%|██████████| 629k/629k [00:00<00:00, 778MB/s]


Extracting archive ./star-type-classification/star-type-classification.zip to ./star-type-classification


In [115]:
import pandas as pd

train = pd.read_csv('/content/star-type-classification/train_star.csv')
test = pd.read_csv('/content/star-type-classification/test_star.csv')
sample_submit = pd.read_csv('/content/star-type-classification/sample_submition_stars.csv')
train

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,9.99,7.92,1.61,0.646,G5/G6V,19.483625,Giant
1,10.86,3.26,2.12,1.840,M1V:,18.426088,Giant
2,8.83,7.57,1.05,0.461,F3V,18.225480,Giant
3,7.72,24.80,0.89,0.613,G3IV,19.692257,Giant
4,8.81,3.17,1.03,0.872,G6III,16.315296,Dwarf
...,...,...,...,...,...,...,...
20727,7.07,14.88,0.90,0.586,F8V,17.933014,Giant
20728,9.11,4.71,0.87,0.485,F5V,17.475105,Giant
20729,9.69,1.61,1.56,0.384,A2/A3IV,15.724129,Giant
20730,8.06,37.33,0.97,0.758,K1V,20.920290,Giant


In [51]:
train.head(200)

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,9.99,7.92,1.61,0.646,G5/G6V,19.483625,Giant
1,10.86,3.26,2.12,1.840,M1V:,18.426088,Giant
2,8.83,7.57,1.05,0.461,F3V,18.225480,Giant
3,7.72,24.80,0.89,0.613,G3IV,19.692257,Giant
4,8.81,3.17,1.03,0.872,G6III,16.315296,Dwarf
...,...,...,...,...,...,...,...
195,5.20,11.55,0.67,-0.059,B9V,15.512910,Giant
196,7.52,17.87,0.97,0.612,G2IV,18.780622,Giant
197,7.86,0.59,0.66,1.570,K3III,11.714260,Dwarf
198,5.81,4.80,0.54,-0.133,B4V,14.216206,Giant


In [ ]:
'''
train['SpType'] = train['SpType'].str.replace(':', '', regex=False)
train['SpType'] = train['SpType'].str.replace('+', '', regex=False)
train['SpType'] = train['SpType'].str.replace('...', '', regex=False)
train['SpType'] = train['SpType'].str.replace('..', '', regex=False)
train['SpType'] = train['SpType'].str.replace(',', '/', regex=False)
train['SpType'] = train['SpType'].str.replace('-', '/', regex=False)
train['SpType'] = train['SpType'].str.replace('e', '', regex=False)
train['SpType'] = train['SpType'].str.replace('comp', '', regex=False)
train['SpType'] = train['SpType'].str.replace('np/', '', regex=False)
train['SpType'] = train['SpType'].str.replace('SB', '', regex=False)
#train['SpType'] = train['SpType'].str.split('/')
train.head(400)
'''

"\ntrain['SpType'] = train['SpType'].str.replace(':', '', regex=False)\ntrain['SpType'] = train['SpType'].str.replace('+', '', regex=False)\ntrain['SpType'] = train['SpType'].str.replace('...', '', regex=False)\ntrain['SpType'] = train['SpType'].str.replace('..', '', regex=False)\ntrain['SpType'] = train['SpType'].str.replace(',', '/', regex=False)\ntrain['SpType'] = train['SpType'].str.replace('-', '/', regex=False)\ntrain['SpType'] = train['SpType'].str.replace('e', '', regex=False)\ntrain['SpType'] = train['SpType'].str.replace('comp', '', regex=False)\ntrain['SpType'] = train['SpType'].str.replace('np/', '', regex=False)\ntrain['SpType'] = train['SpType'].str.replace('SB', '', regex=False)\n#train['SpType'] = train['SpType'].str.split('/')\ntrain.head(400)\n"

In [116]:
import pandas as pd
import re

def clean_sptype(s):
    if s is None:
        return None
    s = str(s).upper()
    s = re.sub(r'[:\+\?EePpNnAaBbSBsb]+', '', s)  # убираем мусор
    return s
def parse_sptype(s):
    if s is None:
        return pd.Series([None, None, None])

    s = clean_sptype(s)

    # 1) Spectral class (extended!)
    m1 = re.search(r'(O|B|A|F|G|K|M|C|N|S|W)', s)
    spec = m1.group(1) if m1 else None

    # 2) Subclass (digits or ranges)
    m2 = re.search(r'(\d(?:/\d)?)', s)
    subclass = m2.group(1) if m2 else None

    # 3) Luminosity class (order matters)
    m3 = re.search(r'(III/IV|IV/V|III|II|IV|V|I)', s)
    lum = m3.group(1) if m3 else None

    return pd.Series([spec, subclass, lum])

def normalize_lum(l):
    if l is None:
        return None
    if '/' in l:
        return l.split('/')[-1]   # берем последнее
    return l

train['lum_class_clean'] = train['lum_class'].apply(normalize_lum)

train.head(400)

KeyError: 'lum_class'

In [110]:
train[train['spec_class'].isna()]

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass,spec_class,subclass,lum_class,lum_class_clean
682,6.22,1.75,0.52,2.673,C5II,12.435190,Dwarf,None,5,II,II
2475,9.14,-2.20,4.98,2.730,NV...,15.852114,Giant,None,None,V,V
5006,5.98,1.72,0.83,0.901,CIb...,12.157642,Dwarf,None,None,I,I
6118,8.50,0.52,1.14,2.860,C6II,12.080017,Dwarf,None,6,II,II
6388,6.30,2.51,0.93,2.349,C5II,13.298368,Dwarf,None,5,II,II
7452,7.89,3.49,1.44,2.450,C6II,15.604128,Dwarf,None,6,II,II
7827,8.96,1.26,1.51,2.928,C5II,14.461853,Dwarf,None,5,II,II
9763,9.08,2.40,1.57,2.765,C4II,15.981056,Dwarf,None,4,II,II
10025,8.23,1.13,1.08,2.420,C5II,13.495392,Dwarf,None,5,II,II
11471,8.27,1.86,1.69,2.523,C4II,14.617565,Dwarf,None,4,II,II


In [102]:
train = train.drop('SpType',axis = 1)
train = train.drop('lum_class',axis = 1)
train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,spec_class,subclass,lum_class_clean
0,9.99,7.92,1.61,0.646,19.483625,Giant,G,5,V
1,10.86,3.26,2.12,1.840,18.426088,Giant,M,1,V
2,8.83,7.57,1.05,0.461,18.225480,Giant,F,3,V
3,7.72,24.80,0.89,0.613,19.692257,Giant,G,3,IV
4,8.81,3.17,1.03,0.872,16.315296,Dwarf,G,6,III
...,...,...,...,...,...,...,...,...,...
20727,7.07,14.88,0.90,0.586,17.933014,Giant,F,8,V
20728,9.11,4.71,0.87,0.485,17.475105,Giant,F,5,V
20729,9.69,1.61,1.56,0.384,15.724129,Giant,A,2,IV
20730,8.06,37.33,0.97,0.758,20.920290,Giant,K,1,V


In [106]:
train[train['spec_class'].isna()]

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,spec_class,subclass,lum_class_clean
682,6.22,1.75,0.52,2.673,12.435190,Dwarf,None,5,II
2475,9.14,-2.20,4.98,2.730,15.852114,Giant,None,None,V
5006,5.98,1.72,0.83,0.901,12.157642,Dwarf,None,None,I
6118,8.50,0.52,1.14,2.860,12.080017,Dwarf,None,6,II
6388,6.30,2.51,0.93,2.349,13.298368,Dwarf,None,5,II
7452,7.89,3.49,1.44,2.450,15.604128,Dwarf,None,6,II
7827,8.96,1.26,1.51,2.928,14.461853,Dwarf,None,5,II
9763,9.08,2.40,1.57,2.765,15.981056,Dwarf,None,4,II
10025,8.23,1.13,1.08,2.420,13.495392,Dwarf,None,5,II
11471,8.27,1.86,1.69,2.523,14.617565,Dwarf,None,4,II


In [95]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown='use_encoded_value',categories = [['O', 'B', 'A', 'F', 'G', 'K', 'M']] ,unknown_value=-1)
encoder.fit(train['spec_class'].values.reshape(-1,1))

encoded = encoder.transform(train['spec_class'].values.reshape(-1,1))
encoded

ValueError: could not convert string to float: 'O'

In [90]:
train['spec_class'] = encoded
train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,spec_class,subclass,lum_class_clean
0,9.99,7.92,1.61,0.646,19.483625,Giant,3.0,5,V
1,10.86,3.26,2.12,1.840,18.426088,Giant,5.0,1,V
2,8.83,7.57,1.05,0.461,18.225480,Giant,2.0,3,V
3,7.72,24.80,0.89,0.613,19.692257,Giant,3.0,3,IV
4,8.81,3.17,1.03,0.872,16.315296,Dwarf,3.0,6,III
...,...,...,...,...,...,...,...,...,...
20727,7.07,14.88,0.90,0.586,17.933014,Giant,2.0,8,V
20728,9.11,4.71,0.87,0.485,17.475105,Giant,2.0,5,V
20729,9.69,1.61,1.56,0.384,15.724129,Giant,0.0,2,IV
20730,8.06,37.33,0.97,0.758,20.920290,Giant,4.0,1,V


In [92]:
encoder2 = OrdinalEncoder(handle_unknown='use_encoded_value',categories=[['I', 'II', 'III', 'IV', 'V']], unknown_value=-1)
encoder2.fit(train['lum_class_clean'].values.reshape(-1,1))

encoded2 = encoder2.transform(train['lum_class_clean'].values.reshape(-1,1))
encoded2

array([[4.],
       [4.],
       [4.],
       ...,
       [3.],
       [4.],
       [3.]])